In [4]:
!pip install git+https://github.com/coreylynch/pyFM

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Cloning https://github.com/coreylynch/pyFM to /tmp/pip-req-build-g7ja8a_o
  Running command git clone -q https://github.com/coreylynch/pyFM /tmp/pip-req-build-g7ja8a_o
  Created wheel for pyfm: filename=pyfm-0.0.0-cp36-cp36m-linux_x86_64.whl size=216161 sha256=3cda63933647847b777eeba9ee984df07b4177531709b12741d1b339ea175b33
  Stored in directory: /tmp/pip-ephem-wheel-cache-wuhm34cw/wheels/4a/ae/53/a68a8862cef4b625f37e35a2879715d21661e3982bac3b42d6
Successfully built pyfm
  Attempting uninstall: pyfm
    Found existing installation: pyfm 0.2.4
    Uninstalling pyfm-0.2.4:
      Successfully uninstalled pyfm-0.2.4
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [181]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.feature_extraction import DictVectorizer
from scipy.sparse import csr_matrix, lil_matrix
from pyfm import pylibfm
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors

In [156]:
initial_train_data = pd.read_json("~/SageMaker/data/dataset_en_train.json", lines=True).sample(frac=1)
initial_test_data = pd.read_json("~/SageMaker/data/dataset_en_test.json", lines=True)

https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html

In [135]:
initial_train_data.describe()

,stars
count,10000.000000
mean,2.969100
std,1.412638
min,1.000000
25%,2.000000
50%,3.000000
75%,4.000000
max,5.000000


In [136]:
initial_train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 54855 to 176624
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   review_id         10000 non-null  object
 1   product_id        10000 non-null  object
 2   reviewer_id       10000 non-null  object
 3   stars             10000 non-null  int64 
 4   review_body       10000 non-null  object
 5   review_title      10000 non-null  object
 6   language          10000 non-null  object
 7   product_category  10000 non-null  object
dtypes: int64(1), object(7)
memory usage: 703.1+ KB


In [137]:
# There are no null values in the train data
initial_train_data.isnull().sum()

review_id           0
product_id          0
reviewer_id         0
stars               0
review_body         0
review_title        0
language            0
product_category    0
dtype: int64

In [164]:
print("Number of unique products are {0}".format(initial_train_data['product_id'].nunique()))
print("Number of unique reviewers are {0}".format(initial_train_data['reviewer_id'].nunique()))
print("Number of unique categories of the product are are {0}".format(initial_train_data['product_category'].nunique()))

Number of unique products are 185541
Number of unique reviewers are 196745
Number of unique categories of the product are are 31


There are so many different unique reviewers and the unique products which leads to a large dimensional space. 

In [157]:
required_data = initial_train_data[['product_id', 'reviewer_id', 'stars']]

In [158]:
data_one_hot_sparse = pd.get_dummies(required_data, columns=['product_id', 'reviewer_id'], sparse = True)

In [159]:
data_one_hot_sparse.head()

,stars,product_id_product_en_0000010,product_id_product_en_0000012,product_id_product_en_0000013,product_id_product_en_0000015,product_id_product_en_0000017,product_id_product_en_0000019,product_id_product_en_0000021,product_id_product_en_0000023,product_id_product_en_0000025,...,reviewer_id_reviewer_en_0999963,reviewer_id_reviewer_en_0999965,reviewer_id_reviewer_en_0999968,reviewer_id_reviewer_en_0999973,reviewer_id_reviewer_en_0999984,reviewer_id_reviewer_en_0999988,reviewer_id_reviewer_en_0999989,reviewer_id_reviewer_en_0999992,reviewer_id_reviewer_en_0999994,reviewer_id_reviewer_en_0999996
146160,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43461,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
90193,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
155812,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29548,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [142]:
required_np = initial_train_data[['product_id', 'reviewer_id']].to_numpy()

rows, r_pos = np.unique(required_np[:,0], return_inverse=True)
cols, c_pos = np.unique(required_np[:,1], return_inverse=True)

pivot_table = np.zeros((len(rows), len(cols)))

MemoryError: Unable to allocate 759. MiB for an array with shape (9957, 9988) and data type float64

In [165]:
reviewers = pd.api.types.CategoricalDtype(initial_train_data['reviewer_id'].unique(), ordered=True)
reviewers = initial_train_data['reviewer_id'].astype('category').cat.codes

products = pd.api.types.CategoricalDtype(initial_train_data['product_id'].unique(), ordered=True)
products = initial_train_data['product_id'].astype('category').cat.codes

In [167]:
sparse_matrix = csr_matrix((initial_train_data['stars'], (reviewers, products)), shape=(initial_train_data['reviewer_id'].nunique(), initial_train_data['product_id'].nunique()))

In [170]:
print(sparse_matrix)

  (0, 42835)	1
  (1, 96484)	5
  (2, 46859)	4
  (3, 31598)	3
  (4, 100659)	4
  (5, 172306)	5
  (6, 140799)	2
  (7, 38013)	5
  (8, 11428)	3
  (9, 98035)	4
  (10, 26392)	3
  (11, 79695)	2
  (12, 61222)	4
  (13, 15055)	2
  (14, 170272)	5
  (15, 148747)	1
  (16, 32718)	4
  (17, 140644)	3
  (18, 126713)	1
  (19, 19731)	3
  (20, 56182)	1
  (21, 69755)	1
  (22, 73272)	1
  (23, 66681)	1
  (24, 120407)	1
  :	:
  (196720, 157207)	5
  (196721, 46579)	2
  (196722, 133219)	4
  (196723, 93305)	3
  (196724, 184182)	4
  (196725, 98681)	4
  (196726, 89588)	5
  (196727, 15150)	3
  (196728, 17212)	2
  (196729, 129301)	5
  (196730, 182954)	5
  (196731, 125354)	4
  (196732, 67635)	5
  (196733, 100516)	5
  (196734, 43907)	4
  (196735, 166071)	3
  (196736, 139333)	2
  (196737, 167484)	5
  (196738, 111537)	3
  (196739, 88731)	2
  (196740, 110269)	2
  (196741, 128492)	5
  (196742, 79076)	4
  (196743, 108558)	4
  (196744, 78116)	2


In [173]:
similarities_sparse = cosine_similarity(sparse_matrix, dense_output=False)

In [194]:
similarities_sparse

<196745x196745 sparse matrix of type '<class 'numpy.float64'>'
	with 229177 stored elements in Compressed Sparse Row format>

In [176]:
# returns index (column position) of top n similarities in each row
def top_n_idx_sparse(matrix, n):
    '''Return index of top n values in each row of a sparse matrix'''
    top_n_idx = []
    for le, ri in zip(matrix.indptr[:-1], matrix.indptr[1:]):
        n_row_pick = min(n, ri - le)
        top_n_idx.append(matrix.indices[le + np.argpartition(matrix.data[le:ri], -n_row_pick)[-n_row_pick:]])
    return top_n_idx

user_user_similar = top_n_idx_sparse(sparse_matrix, 5)

In [192]:
user_user_similar

[array([42835], dtype=int32),
 array([96484], dtype=int32),
 array([46859], dtype=int32),
 array([31598], dtype=int32),
 array([100659], dtype=int32),
 array([172306], dtype=int32),
 array([140799], dtype=int32),
 array([38013], dtype=int32),
 array([11428], dtype=int32),
 array([98035], dtype=int32),
 array([26392], dtype=int32),
 array([79695], dtype=int32),
 array([61222], dtype=int32),
 array([15055], dtype=int32),
 array([170272], dtype=int32),
 array([148747], dtype=int32),
 array([32718], dtype=int32),
 array([140644], dtype=int32),
 array([126713], dtype=int32),
 array([19731], dtype=int32),
 array([56182], dtype=int32),
 array([69755], dtype=int32),
 array([73272], dtype=int32),
 array([66681], dtype=int32),
 array([120407], dtype=int32),
 array([8016], dtype=int32),
 array([93091], dtype=int32),
 array([116800], dtype=int32),
 array([126179], dtype=int32),
 array([155724], dtype=int32),
 array([73879], dtype=int32),
 array([38929], dtype=int32),
 array([110436], dtype=int32),

In [177]:
user_user_similar_dict = {}
for idx, val in enumerate(user_user_similar):
        user_user_similar_dict.update({idx: val.tolist()})

In [178]:
similar_users_final = {}
for user, similar_users in user_user_similar_dict.items():
    idx = reviewers.index[user]
    values = []
    for value in similar_users:
        values.append(reviewers.index[value])

    similar_users_final.update({idx: values})

In [179]:
similar_users_final

{146160: [93490],
 43461: [79564],
 90193: [163193],
 155812: [119075],
 29548: [2095],
 70953: [80621],
 151603: [165104],
 83876: [179570],
 142769: [139962],
 181911: [175160],
 5698: [16266],
 105680: [19659],
 177544: [105561],
 22586: [119013],
 33145: [34374],
 143368: [4649],
 143170: [80494],
 162800: [1934],
 127902: [126060],
 139195: [2625],
 152983: [38237],
 163549: [38286],
 144610: [110417],
 139588: [73607],
 30372: [44819],
 140543: [146570],
 72131: [68759],
 58646: [40245],
 21804: [84510],
 104840: [189355],
 33971: [144345],
 55519: [2257],
 170277: [28574],
 163404: [46813],
 187254: [61041],
 166341: [111065],
 123156: [79874],
 199168: [120899],
 18118: [184687],
 43204: [39096],
 11917: [51992],
 106643: [169583],
 144890: [5082],
 157754: [44658],
 1325: [166449],
 47354: [93695],
 158097: [22151],
 135504: [21471],
 153788: [139585],
 172303: [194457],
 22100: [92993],
 159199: [23781],
 185593: [158487],
 166977: [97150],
 81215: [51575],
 118580: [135448],

In [183]:
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(sparse_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [190]:
query_index = np.random.choice(reviewers.shape[0])


97887

In [191]:
distances, indices = model_knn.kneighbors(reviewers.iloc[query_index].reshape(1, -1), n_neighbors = 6)

ValueError: Incompatible dimension for X and Y matrices: X.shape[1] == 1 while Y.shape[1] == 185541

In [113]:
transformed_pd = initial_train_data[['reviewer_id', 'product_id']]
dict_pd = transformed_pd.to_dict('records')

y_train = initial_train_data['stars'].to_numpy()
y_train = y_train.astype("double")

In [114]:
transformed_pd = initial_test_data[['reviewer_id', 'product_id']]
dict_test_pd = transformed_pd.to_dict('records')

y_test = initial_test_data['stars'].to_numpy()
y_test = y_test.astype("double")

In [131]:
reviewer_product_interaction = pd.pivot_table(initial_train_data[['stars', 'reviewer_id', 'product_id']], index='reviewer_id', columns='product_id', values='stars')

MemoryError: Unable to allocate 760. MiB for an array with shape (9994, 9971) and data type float64

In [126]:
reviewer_product_interaction.isnull().sum()

product_id
product_en_0000543    1999
product_en_0001087    1999
product_en_0001114    1999
product_en_0001868    1999
product_en_0002471    1999
                      ... 
product_en_0997719    1999
product_en_0997887    1999
product_en_0999064    1999
product_en_0999165    1999
product_en_0999957    1999
Length: 1999, dtype: int64

In [77]:
v = DictVectorizer()
X_train = v.fit_transform(dict_pd)
#print(X_train)
v.get_feature_names()[1]
#X_test = v.transform(dict_test_pd)

'product_id=product_en_0000012'

In [69]:
fm = pylibfm.FM(num_factors=1, num_iter=5, verbose=True, task="regression", initial_learning_rate=0.001, learning_rate_schedule="optimal")

In [70]:
fm.fit(X_train, y_train)

Creating validation dataset of 0.01 of training for adaptive regularization
-- Epoch 1
Training log loss: 0.49936
-- Epoch 2
Training log loss: 0.49458
-- Epoch 3
Training log loss: 0.49376
-- Epoch 4
Training log loss: 0.49326
-- Epoch 5
Training log loss: 0.49282


In [65]:
fm.predict(X_test)

array([0.20317445, 0.20317445, 0.20317445, ..., 0.20382012, 0.20317445,
       0.20317445])

In [ ]:
user_item_matrix = np.zeros((int(initial_train_data['reviewer_id'].nunique()), initial_train_data['product_id'].nunique()))

This is not sustainable as we cannot have that much memory.

In [ ]:
# I am assuming that the product_id, review_id, reviewer_id, product and language are irrelevant for finding out the product
# So instead of dropping the columns I can keep only relevant data
initial_training_data = initial_train_data[['review_body','stars']]
initial_training_data.head()

In [ ]:
# Same can be done with test data
initial_test_data = initial_test_data[['review_body','stars', 'review_title']]

In [ ]:
# Remove the product_category as it is the target item.
X = initial_train_data.drop(['stars'], axis=1)
y = initial_train_data['stars']

# Same for the test data as well.
X_test = initial_test_data.drop(['stars'], axis=1)
y_test = initial_test_data['stars']

https://towardsdatascience.com/1-to-5-star-ratings-classification-or-regression-b0462708a4df

In [ ]:
# Now we have to use word vectorization 
count_vect = CountVectorizer()

In [ ]:
X_train_counts = count_vect.fit_transform(X['review_body'])
X_train_counts.shape

In [ ]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

In [ ]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB().fit(X_train_tfidf, y)

In [ ]:
X_test_counts = count_vect.transform(X_test['review_body'])
X_test_tfidf = tfidf_transformer.transform(X_test_counts)
X_test_tfidf.shape

In [ ]:
import pickle

file = open('model.pkl','wb')
pickle.dump(model, file)

In [ ]:
predicted = model.predict(X_test_tfidf)

In [ ]:
np.mean(predicted == y_test)